In [1]:
import aws_cdk as cdk
from aws_cdk import aws_secretsmanager as secretsmanager

from infra.utils.environments import environments
from infra.lib.foundations import FoundationsStack, FoundationsStackProps
from infra.lib.etl import EtlStack, EtlStackProps
from infra.lib.data_migration import DataMigrationStack, DataMigrationStackProps
from infra.lib.analytics_consumption import AnalyticsConsumptionStack, AnalyticsConsumptionStackProps
from infra.utils.naming import create_name

app = cdk.App()

# Get context value from CLI: cdk deploy -c config=dev
# config = app.node.try_get_context("config")
# if not config:
#     raise ValueError("Context variable missing on CDK command. Pass in as '-c config=XXX'")
config = 'dev'

context_env = environments[config]
region = context_env.region
environment = context_env.environment
account_id = context_env.accountId
project = context_env.project

b'{"stderr":"W1dBUk5JTkddIGF3cy1jZGstbGliLmF3c19keW5hbW9kYi5UYWJsZU9wdGlvbnMjcG9pbnRJblRpbWVSZWNvdmVyeSBpcyBkZXByZWNhdGVkLgogIHVzZSBgcG9pbnRJblRpbWVSZWNvdmVyeVNwZWNpZmljYXRpb25gIGluc3RlYWQKICBUaGlzIEFQSSB3aWxsIGJlIHJlbW92ZWQgaW4gdGhlIG5leHQgbWFqb3IgcmVsZWFzZS4K"}\n'
b'{"stderr":"W1dBUk5JTkddIGF3cy1jZGstbGliLmF3c19keW5hbW9kYi5UYWJsZU9wdGlvbnMjcG9pbnRJblRpbWVSZWNvdmVyeSBpcyBkZXByZWNhdGVkLgogIHVzZSBgcG9pbnRJblRpbWVSZWNvdmVyeVNwZWNpZmljYXRpb25gIGluc3RlYWQKICBUaGlzIEFQSSB3aWxsIGJlIHJlbW92ZWQgaW4gdGhlIG5leHQgbWFqb3IgcmVsZWFzZS4K"}\n'
b'{"stderr":"W1dBUk5JTkddIGF3cy1jZGstbGliLmF3c19keW5hbW9kYi5UYWJsZU9wdGlvbnMjcG9pbnRJblRpbWVSZWNvdmVyeSBpcyBkZXByZWNhdGVkLgogIHVzZSBgcG9pbnRJblRpbWVSZWNvdmVyeVNwZWNpZmljYXRpb25gIGluc3RlYWQKICBUaGlzIEFQSSB3aWxsIGJlIHJlbW92ZWQgaW4gdGhlIG5leHQgbWFqb3IgcmVsZWFzZS4K"}\n'
b'{"stderr":"W1dBUk5JTkddIGF3cy1jZGstbGliLmF3c19zdGVwZnVuY3Rpb25zLlN0YXRlTWFjaGluZVByb3BzI2RlZmluaXRpb24gaXMgZGVwcmVjYXRlZC4KICB1c2UgZGVmaW5pdGlvbkJvZHk6IERlZmluaXRpb25Cb2R5LmZyb21DaGFpbmFibGUoKQogIF

In [2]:
stack_foundations = FoundationsStack(
    app,
    create_name("stack", "foundations"),
    env=cdk.Environment(account=account_id, region=region),
    props=FoundationsStackProps(context_env=context_env),
)

In [3]:
redshift_secret = secretsmanager.Secret.from_secret_name_v2(
    stack_foundations,
    "RedshiftSecretImported",
    "dl-redshift-admin-dev-secrets"  # tu secreto
)
redshift_secret_arn = redshift_secret.secret_arn

In [4]:
stack_etl = EtlStack(
    app,
    create_name("stack", "etl"),
    env=cdk.Environment(account=account_id, region=region),
    props=EtlStackProps(
        context_env=context_env,
        storage=stack_foundations.storage,
        metadata_catalog=stack_foundations.metadata_catalog,
        security=stack_foundations.security,
        raw_database_name=stack_foundations.governance.raw_database_name,
        master_database_name=stack_foundations.governance.master_database_name,
        analytics_database_name=stack_foundations.governance.analytics_database_name,
        datalake_lib_layer=stack_foundations.datalake_layer,
        redshift_jdbc_url=None,
        redshift_secret_arn=redshift_secret_arn,
    ),
)